This is your private sketch-notebook. Do whatever you want with it. When code is ready to be developed, please just copy and paste it to the final notebook. 

In [2]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  
from PIL import Image
import glob
import numpy as np
import pandas as pd
from IPython.display import Image
from IPython.display import Image
from IPython.display import Image
from IPython.core.display import HTML 


Using TensorFlow backend.


In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:
# the folder path to the folder with all the fishies in there
folders_path = '../Data/Kaggle/train/*'

# creating a dataframe with the columns below
image_dict = {}

# going through all folders and saving file names in dataframe so that order is always correct
for foldername in glob.glob(folders_path):
    for image_name in glob.glob(foldername + '/*.jpg'):
        # getting image name and fish type and putting it in a dictionary
        fish_type = foldername[-3:]
        image_dict[image_name] = fish_type

image_df = pd.DataFrame(image_dict.items(), columns = ['image_name', 'fish_type']).sample(frac=1) # random sampling
dummy_df = pd.get_dummies(image_df, columns=['fish_type']) # dummify data

dummy_df.head()

,image_name,fish_type_ALB,fish_type_ARK,fish_type_BET,fish_type_DOL,fish_type_HER,fish_type_LAG,fish_type_NoF,fish_type_YFT
2035,../Data/Kaggle/train/YFT/img_05451.jpg,0,0,0,0,0,0,0,1
532,../Data/Kaggle/train/SHARK/img_07056.jpg,0,1,0,0,0,0,0,0
3515,../Data/Kaggle/train/YFT/img_01064.jpg,0,0,0,0,0,0,0,1
3141,../Data/Kaggle/train/NoF/img_00849.jpg,0,0,0,0,0,0,1,0
3074,../Data/Kaggle/train/DOL/img_07212.jpg,0,0,0,1,0,0,0,0


In [5]:
# Smaller dataset for testing purposes!
dummy_df = dummy_df.iloc[0:101,:]

In [6]:
Image(url= "https://kaggle2.blob.core.windows.net/competitions/kaggle/5568/media/species-ref-key.jpg")

In [7]:
# Count of the image types
image_df.groupby('fish_type').count().sort_values('image_name')

,image_name
fish_type,
LAG,67
DOL,117
ARK,176
BET,200
HER,299
NoF,465
YFT,734
ALB,1719


In [8]:
count = 0 # count, to test the algorithm with 100 pictures
images = []

for filename in dummy_df.image_name:
    img = cv2.imread(filename) # Read file    
    img = cv2.resize(img, (224, 224)) # Resize image to 224 x 224 pixels (dont feel obliged to use this)
    img = img.astype(np.float32) # Change to 32 bit floats (good for neural networks, other floats not as much making sense)

    # "Normalize" images
    for rgb_value in [0,1,2]:
        img[:, :, rgb_value] = img[:, :, rgb_value] / 255.0 # 255 is the RGB maximum
        img[:, :, rgb_value] -= numpy.mean(img[:, :, rgb_value])

    img = img.transpose((2,0,1))

    images.append(img)

# X and y value and labels
x = np.asarray(images)
y = dummy_df.iloc[:,1:9].as_matrix()
labels = dummy_df.image_name.apply(lambda path: path[-13:])

# table, just to be sure that everything goes well
dummy_df['image_bytes'] = images
dummy_df.head()

,image_name,fish_type_ALB,fish_type_ARK,fish_type_BET,fish_type_DOL,fish_type_HER,fish_type_LAG,fish_type_NoF,fish_type_YFT,image_bytes
2035,../Data/Kaggle/train/YFT/img_05451.jpg,0,0,0,0,0,0,0,1,"[[[0.261446, 0.257524, 0.253603, 0.245759, 0.2..."
532,../Data/Kaggle/train/SHARK/img_07056.jpg,0,1,0,0,0,0,0,0,"[[[-0.22421, -0.216367, -0.22421, -0.220288, -..."
3515,../Data/Kaggle/train/YFT/img_01064.jpg,0,0,0,0,0,0,0,1,"[[[0.047504, 0.118092, 0.122014, 0.157308, 0.2..."
3141,../Data/Kaggle/train/NoF/img_00849.jpg,0,0,0,0,0,0,1,0,"[[[0.612175, 0.600411, 0.608254, 0.604332, 0.5..."
3074,../Data/Kaggle/train/DOL/img_07212.jpg,0,0,0,1,0,0,0,0,"[[[0.562012, 0.550248, 0.558091, 0.554169, 0.5..."


In [9]:
split_no = 90

x_train = x[:split_no]
y_train = y[:split_no]
labels_train = labels[:split_no]

x_test = x[split_no:]
y_test = y[split_no:]
labels_test = labels[split_no:]

In [10]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 224, 224), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
# Compile model
epochs = 1 # dont leave it like this, only for testing!
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 224, 224)  896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 32, 224, 224)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 224, 224)  9248        dropout_1[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 112, 112)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [11]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
classes = model.predict_proba(x_test, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print("Accuracy: %.2f%%" % (scores[1]*100))
print ' '
proba_df

Train on 90 samples, validate on 11 samples
Epoch 1/1
11/11 [==============================] - 0s
Accuracy: 36.36%
 


fish_type,ALB,ARK,BET,DOL,HER,LAG,NoF,YFT
0,0.426173,0.065238,0.086478,0.035540,0.112373,0.021211,0.060413,0.192574
1,0.427818,0.068423,0.076040,0.054652,0.121715,0.039472,0.073129,0.138751
2,0.464760,0.064145,0.069278,0.051579,0.118718,0.037686,0.063735,0.130099
3,0.422395,0.067694,0.080242,0.050241,0.133976,0.030940,0.068498,0.146015
4,0.408787,0.071411,0.082740,0.052695,0.125049,0.040214,0.072238,0.146866
5,0.238855,0.106769,0.124092,0.077254,0.121389,0.060475,0.098026,0.173141
6,0.441850,0.063033,0.067672,0.051304,0.118149,0.033770,0.070237,0.153986
7,0.401539,0.075477,0.081405,0.044323,0.137396,0.027657,0.058664,0.173539
8,0.329677,0.085461,0.098731,0.074396,0.127174,0.053225,0.084162,0.147175
9,0.288342,0.087232,0.098668,0.082990,0.123380,0.058878,0.095088,0.165423


In [12]:
proba_df['img_names'] = labels_test.values